In [3]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import cv2 as cv
import zipfile

In [4]:
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from object_detection.utils import ops as utils_ops

In [5]:
if StrictVersion (tf.__version__) < StrictVersion('1.12.0'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')

In [6]:
print(tf.__version__)

2.7.0


In [7]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [8]:
import argparse 

In [9]:
def parse_arg():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_name', required=True, help='Path to model')
    parser.add_argument('--label_map', required = True, help='Path to label map text')
    parser.add_argument('--test_annot_text', required=True, help='Path to test annotation text')
    parser.add_argument('--test_image_dir', required=True, help='Path to test image directory')
    parser.add_argument('--output_dir', required=True, help='Path to output directory')
    return parser.parse_args()

In [10]:
def load_image_intro_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
    (im_height, im_width, 3)).astype(np.uint8)

In [13]:
def run_interfence_for_single_image(image,graph):
    with graph.as_default():
        with tf.Session() as sees:
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict={}
        for key in = [
            'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
        ]:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                tensor_name)
            
            if 'detection_masks' in tensor_dict:
                 # The following processing is only for single image
            detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
            detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
            
            real_num_detection = tf.cast(tensor_dict['num_detection'][0].tf.int32)
            detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
            detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
            detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                detection_masks, detection_boxes, image.shape[1], image.shape[2])
            detection_masks_reframed = tf.cast(
                tf.greater(detection_masks_reframed, 0.5), tf.uint8)

IndentationError: expected an indented block (Temp/ipykernel_852/355179883.py, line 4)